In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)

c:\Users\Camille\Documents\TWR


## Instanciando repositories

In [2]:
from training_models.config.database import AsyncSessionLocal
from training_models.repositories.campaigns_repository import CampaignRepository
from training_models.repositories.request_repository import RequestsRepository
from training_models.config.mongo import mongo_collection, mongo_client

campaign_repository = CampaignRepository(session_factory=AsyncSessionLocal)
mongo_request_repository = RequestsRepository(collection=mongo_collection)

Garantindo índices...


c:\Users\Camille\Documents\TWR\mil\.venv\Lib\site-packages\motor\core.py:171: UserWarning: You appear to be connected to a DocumentDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/documentdb
  delegate = self.__delegate_class__(*args, **kwargs)


In [3]:
DATA_PATH = "G:/Meu Drive/TWR/data"

In [4]:
traffic_sources = [
      "google", 
      "taboola",
      "tiktok",
      "kwai",
      "facebook",
      
]

## Treinamento fasttext

In [5]:
# from src.training_models.services.fasttext_service import FastTextTrainingService
# from tqdm import tqdm

# for traffic_source in tqdm(traffic_sources, desc="Treinando modelos FastText para cada fonte de tráfego"):

#       fasttext_service = FastTextTrainingService(
#             traffic_source=traffic_source, 
#             repo_requests=mongo_request_repository,
#             repo_campaigns=campaign_repository
#       )
      
#       await fasttext_service.train_pipeline(
#             vector_size=300,
#             epochs=10,
#             save_path=F"{DATA_PATH}/{traffic_source}/fasttext_{traffic_source}.model"
#       )

In [ ]:
import torch
from training_models.services.training_service import MILTrainingService

try:
      for source in traffic_sources:
            print(f"\n{'='*50}")
            print(f"Iniciando Pipeline de Treinamento: {source.upper()}")
            print(f"{'='*50}")

            try:
                  training_service = MILTrainingService(
                        traffic_source=source,
                        repo_requests=mongo_request_repository,
                        repo_campaigns=campaign_repository,
                        device="cuda" if torch.cuda.is_available() else "cpu",
                        emb_config="fasttext",
                  )

                  await training_service.training_pipeline(
                        epochs=15,
                  )
                  print(f"Pipeline para '{source}' finalizado com sucesso!")

            except Exception as loop_err:
                  print(f"Ocorreu um erro durante o treinamento de '{source}': {loop_err}")

except Exception as e:
      
      print(f"Ocorreu um erro fatal geral: {e}")


c:\Users\Camille\Documents\TWR\mil\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using 11 out of 12 cores

Iniciando Pipeline de Treinamento: GOOGLE
2026-02-26 12:31:09,699 - INFO - Inicializando MILTrainingService...
2026-02-26 12:31:09,699 - INFO - Dispositivo configurado: cpu
2026-02-26 12:31:09,699 - INFO - Definindo instância do embedding...
2026-02-26 12:31:09,699 - INFO - Carregando instância de embedding (fasttext) do caminho: G:/Meu Drive/TWR/data/google/fasttext_google.model...
Fastext model path:  G:/Meu Drive/TWR/data/google/fasttext_google.model
[DEBUG] Model Path FASTTEXT: G:/Meu Drive/TWR/data/google/fasttext_google.model
2026-02-26 12:31:37,417 - INFO - Dimensão do embedding configurada: 300
2026-02-26 12:31:37,418 - INFO - Iniciando pipeline de treinamento MIL com Atenção...
2026-02-26 12:31:37,419 - INFO - Iniciando geração de embeddings e criação de Bags (MIL)...
2026-02-26 12:31:37,420 - INFO - Buscando campanhas ativas recentes para a fonte: google...
2026-02-26 12:31:41,526 - INFO - Encontradas 100 campanhas. Buscando amostras de requests...
[

Criando Vocabulário: 100%|██████████| 20000/20000 [00:09<00:00, 2113.95it/s]


Using 11 out of 12 cores


Vetorizando: 100%|██████████| 20000/20000 [06:16<00:00, 53.19it/s] 


...

Using 11 out of 12 cores


Vetorizando: 100%|██████████| 20000/20000 [04:07<00:00, 80.93it/s] 


Finishing encoding
2026-02-26 13:11:37,533 - INFO - Agrupando requisições em Bags por Endereço IP...
2026-02-26 13:11:38,918 - INFO - Total de IPs (Bags) formados: 13681
2026-02-26 13:11:38,919 - INFO - Dividindo Bags em treino (80%) e teste (20%)...
2026-02-26 13:11:38,953 - INFO - Criando MILBagDatasetLogical...
2026-02-26 13:11:41,049 - INFO - Instanciando DataLoaders (batch_size=1 para MIL)...
2026-02-26 13:11:41,070 - INFO - Geração de embeddings e DataLoaders concluída.
2026-02-26 13:11:41,106 - INFO - Instanciando a rede neural AttentionMIL...
2026-02-26 13:11:41,153 - INFO - Treinando por 15 épocas...


c:\Users\Camille\Documents\TWR\mil\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


2026-02-26 13:12:41,481 - INFO - Época 1/15 - Train Loss: 0.4701
2026-02-26 13:13:38,179 - INFO - Época 2/15 - Train Loss: 0.4091


In [ ]:
# from src.training_models.services.training_service import TraingingService

# try:
#       for source in traffic_sources:
#             print(f"\n{'='*50}")
#             print(f"Iniciando Pipeline de Treinamento: {source.upper()}")
#             print(f"{'='*50}")

#             try:
#                   training_service = TraingingService(
#                         traffic_source=source,
#                         repo_requests=mongo_request_repository,
#                         repo_campaigns=campaign_repository,
#                         emb_config="transformers"
#                   )

#                   await training_service.training_pipeline(
#                         epochs=10,
#                         burn_in=0.5,
#                         path = DATA_PATH
#                   )
#                   print(f"Pipeline para '{source}' finalizado com sucesso!")

#             except Exception as loop_err:
#                   print(f"Ocorreu um erro durante o treinamento de '{source}': {loop_err}")
#                   print(f"⏭Pulando para a próxima fonte...")

# except Exception as e:
#       print(f"Ocorreu um erro fatal geral: {e}")

# # finally:
# #       # Desligamento Gracioso (Graceful Shutdown) para evitar o erro de "Event loop closed"
# #       print("\nEncerrando conexões com o banco de dados...")
      
# #       # Fecha MySQL
# #       if hasattr(engine, 'dispose'):
# #             await engine.dispose()
# #       print("Conexão MySQL/SQLAlchemy fechada.")
      
# #       # Fecha MongoDB
# #       if mongo_client:
# #             mongo_client.close()
# #       print("Conexão MongoDB fechada.")